In [2]:
print('hello')

hello


In [ ]:
import openai
from dotenv import load_dotenv
import os



In [3]:
load_dotenv()

# Now you can access your environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.chat_models import ChatOpenAI
import requests

db_url = os.getenv('Workout_DB_URL')
# Function to fetch a random cat fact
def get_cat_fact(_: str) -> str:
    try:
        response = requests.get("https://catfact.ninja/fact")
        if response.status_code == 200:
            data = response.json()
            return data.get("fact", "Couldn't find a cat fact.")
        return f"Failed to get cat fact. Status code: {response.status_code}"
    except Exception as e:
        return f"API call failed: {e}"
def get_user_workout_history(_: str) -> str:
    # Simulate fetching user workout history
    return "User workout history: 5 workouts in the last month."
def update_calendar(event_details: str, event_time: str, event_date: str) -> str:
    # Simulate updating a calendar
    print(f"Updating calendar with event: {event_details} at {event_time} on {event_date}")
    return f"Calendar updated with event: {event_details}"
def log_workout(workout_details: str) -> str:
    # Simulate logging a workout
    return f"Workout logged: {workout_details}"

# Wrap it as a LangChain Tool
cat_fact_tool = Tool(
    name="CatFactFetcher",
    func=get_cat_fact,
    description="Provides a random cat fact when someone asks for one."
)
update_calendar_tool = Tool(
    name="UpdateCalendar",
    func=update_calendar,
    description="Updates the calendar with the provided information, takes event details, time, and date as input."
)
grab_user_workout_history = Tool(
    name="GrabUserWorkoutHistory",
    func=get_user_workout_history,
    description="Fetches the user's workout history."
)


# Set up the agent
llm = ChatOpenAI(temperature=.2)
agent = initialize_agent(
    tools=[cat_fact_tool, update_calendar_tool, grab_user_workout_history],
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage
print(agent.run("Hi tell me a cat fact and put it in my calendar for next Monday at 10am, let me know when it's done."))




> Entering new AgentExecutor chain...
We need to fetch a cat fact and update the calendar with the event details for next Monday at 10am.
Action: CatFactFetcher
Action Input: _
Observation: Cats have supersonic hearing
Thought:We have the cat fact, now we need to update the calendar.
Action: UpdateCalendar
Action Input: "Cat Fact Event", "10:00 AM", "Monday"

TypeError: update_calendar() missing 2 required positional arguments: 'event_time' and 'event_date'

In [ ]:
from langchain.chains import RetrievalQA
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
import pandas as pd

# 1. Load Excel
df = pd.read_excel("climbing_training.xlsx")

# 2. Convert rows into text chunks
docs = []
for _, row in df.iterrows():
    text = f"Ability: {row['Ability Level']}, Exercise: {row['Exercise']}, Sets: {row['Sets']}, Reps: {row['Reps/Time']}, Notes: {row['Notes']}"
    docs.append(text)

# 3. Create embeddings + vector store
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_texts(docs, embeddings)

# 4. Build QA chain
qa = RetrievalQA.from_chain_type(
    llm=OpenAI(),
    retriever=vectorstore.as_retriever(),
    chain_type="stuff"
)

# 5. Example user query
query = "Create a 6-week plan for a V3 climber aiming to reach V5"
response = qa.run(query)
print(response)

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent, Tool
from langchain.agents.agent_types import AgentType
from langchain.tools import StructuredTool
from pydantic import BaseModel

# Define argument schemas using Pydantic
class CalendarInput(BaseModel):
    event_details: str
    event_time: str
    event_date: str

def update_calendar(event_details: str, event_time: str, event_date: str) -> str:
    return f"Calendar updated: {event_details} at {event_time} on {event_date}"

# Wrap with StructuredTool to expose argument schema
update_calendar_tool = StructuredTool.from_function(
    func=update_calendar,
    args_schema=CalendarInput,
    name="UpdateCalendar",
    description="Updates the calendar with event details, time, and date"
)

def get_cat_fact() -> str:
    return "Cats sleep for 70% of their lives!"

cat_fact_tool = StructuredTool.from_function(
    func=get_cat_fact,
    name="CatFactFetcher",
    description="Returns a random cat fact. No input required."
)

tools = [update_calendar_tool, cat_fact_tool]

llm = ChatOpenAI(temperature=0)

agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

# Example usage:
print(agent.run("Add a cat fact to my calendar for next Monday at 10am."))




> Entering new AgentExecutor chain...
Thought: I need to add a cat fact to the user's calendar for next Monday at 10am. I'll use the CatFactFetcher tool to get a random cat fact.

Action:
```
{
  "action": "CatFactFetcher",
  "action_input": {}
}
```


Observation: Cats sleep for 70% of their lives!
Thought:I have fetched a cat fact for you. Now, I will update your calendar with this cat fact for next Monday at 10am.

Action:
```
{
  "action": "UpdateCalendar",
  "action_input": {
    "event_details": "Cat Fact: Cats sleep for 70% of their lives!",
    "event_time": "10:00 AM",
    "event_date": "2023-10-02"
  }
}
```
Observation: Calendar updated: Cat Fact: Cats sleep for 70% of their lives! at 10:00 AM on 2023-10-02
Thought:I have successfully added the cat fact to your calendar for next Monday at 10am.

> Finished chain.
I have successfully added the cat fact to your calendar for next Monday at 10am.
